# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [29]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [30]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [31]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [32]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [33]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [34]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [35]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [36]:
create_table1 = """
CREATE TABLE IF NOT EXISTS music_by_session_and_items 
    (artist text, first_name text, gender text, item_in_session int, last_name text, length double, level text, location text, session_id text, song text, user_id int, PRIMARY KEY(session_id, item_in_session))"""

create_table2 = """
CREATE TABLE IF NOT EXISTS music_history_by_user_and_session
    (artist text, song text, user text, user_id int, session_id text, item_in_session int, PRIMARY KEY(user_id, session_id, item_in_session))
"""

create_table3 = """
CREATE TABLE IF NOT EXISTS users_by_song
    (user text, song text, PRIMARY KEY(song))
"""

create_queries = [create_table1, create_table2, create_table3]
try:
    for create_table in create_queries:
        session.execute(create_table)
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [37]:
query1 = "SELECT artist, song, length FROM music_by_session_and_items WHERE session_id = '{0}' AND item_in_session = {1}"

In [38]:
file = 'event_datafile_new.csv'

def insert_query1(session, row):
    query = "INSERT INTO music_by_session_and_items (artist, first_name, gender, item_in_session, last_name, length, level, location, session_id, song, user_id)"
    query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) "
    session.execute(query, (row['artist'], row['firstName'], row['gender'], int(row['itemInSession']), row['lastName'], float(row['length']), row['level'], row['location'], str(row['sessionId']), row['song'], row['userId']))

def insert_query2(session, row):
    query = "INSERT INTO music_history_by_user_and_session (artist, song, user, user_id, session_id, item_in_session)"
    query = query + " VALUES (%s, %s, %s, %s, %s, %s) "
    user_name = row['firstName'] + " " + row['lastName']
    session.execute(query, (row['artist'], row['song'], user_name, row['userId'], str(row['sessionId']), row['itemInSession']))

def insert_query3(session, row):
    query = "INSERT INTO users_by_song (user, song)"
    query = query + " VALUES (%s, %s) "
    user_name = row['firstName'] + " " + row['lastName']
    session.execute(query, (user_name, row['song']))

file_data = pd.read_csv(file)
for index, row in file_data.iterrows():
    insert_query1(session, row)
    insert_query2(session, row)
    insert_query3(session, row)

#### Do a SELECT to verify that the data have been inserted into each table

In [39]:
check_music_app_history_query = "SELECT * FROM music_by_session_and_items LIMIT 2"
try:
    rows = session.execute(check_music_app_history_query)
    for row in rows:
        print(row)
except Exception as e:
    print(e)

Row(session_id='631', item_in_session=0, artist='Pavement', first_name='Kynnedi', gender='F', last_name='Sanchez', length=99.16036, level='free', location='Cedar Rapids, IA', song='Mercy:The Laundromat', user_id=89)
Row(session_id='403', item_in_session=0, artist='Parachute', first_name='Dominick', gender='M', last_name='Norris', length=146.05016, level='free', location='Los Angeles-Long Beach-Anaheim, CA', song='She Is Love', user_id=45)


In [40]:
try:
    rows = session.execute(query1.format("338",4))
    for row in rows:
        print(row)
except Exception as e:
    print(e)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.3073)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [41]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = "SELECT artist, song, user FROM music_history_by_user_and_session WHERE user_id = {0} AND session_id = '{1}'"
# Query 1 execution
try:
    rows = session.execute(query2.format(10,'182'))
    for row in rows:
        print(row)
except Exception as e:
    print(e)
                    

Row(artist='Down To The Bone', song="Keep On Keepin' On", user='Sylvie Cruz')
Row(artist='Three Drives', song='Greece 2000', user='Sylvie Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', user='Sylvie Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user='Sylvie Cruz')


In [42]:
query3 = "SELECT user FROM users_by_song WHERE song = '{0}'"

try:
    rows = session.execute(query3.format("All Hands Against His Own"))
    for row in rows:
        print(row)
except Exception as e:
    print(e)         

Row(user='Sara Johnson')


### Drop the tables before closing out the sessions

In [44]:
drop_table1 = "DROP TABLE music_by_session_and_items"
drop_table2 = "DROP TABLE music_history_by_user_and_session"
drop_table3 = "DROP TABLE users_by_song"
drop_tables = [drop_table1, drop_table2, drop_table3]
for drop_table in drop_tables:
    try:
        session.execute(drop_table)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [45]:
session.shutdown()
cluster.shutdown()